In [5]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.stats as scipy
sys.path.append("..\\backtesting")
import dynamic_backtesting as db
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["grey", "green", "black", "deepskyblue", "orange", "lime", "purple", "Gold", "brown", "blue", "pink"]) 


In [6]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
asset_names = list(assets.values())

**Henter return data udfra tickers**

In [7]:
return_data =db.download_return_data(tickers, save_to_csv=False)

[*********************100%***********************]  11 of 11 completed


**Henter MGARCH model fit**

In [8]:
coef, residuals, sigmas = db.fit_garch_model(250*2)
mu, o, al, be, dcca, dccb = db.parse_garch_coef(coef, 11)

NotImplementedError: Conversion 'py2rpy' not defined for objects of type '<class 'tuple'>'

**Simulere Non central t-fordelinger til $r_t$. Bemærk at disse ikke er dem fra MGARCH modellen**

In [99]:
#fitter NCT fordeling til return data
t_dist= return_data.apply(scipy.nct.fit, axis=0) 
stat = pd.DataFrame([t_dist.iloc[0],t_dist.iloc[1],t_dist.iloc[2],t_dist.iloc[3]], columns = tickers, index = ['DF','NC','LOC','SCALE']).transpose()

#Sætter længde af simuleringen og random state
T=len(return_data)
state=2016

#Simulere data og ligger det i et dataframe
for index,row in stat.iterrows():
    simulated_returns_1=scipy.nct.rvs(row['DF'],row['NC'],loc=row['LOC'],scale=row['SCALE'],size=T,random_state=state)
    if index=="EEM":
        simulated_returns=pd.DataFrame(simulated_returns_1, columns = ["EEM"])
    else:
        simulated_returns[index]=simulated_returns_1

**Udregner $\Omega_{t+1}$ via simuleret data**

In [100]:
Omega_ts=db.calc_Omega_ts(simulated_returns,sigmas,residuals,dcca, dccb, o, al, be,  mu )

**Udregner vægte**

In [101]:
v_t=db.calc_weights_garch_no_trading_cost(Omega_ts)

**sammenligner 1/N portfølge**